In [4]:
import requests as re
res = re.get("https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2025-11-01").json()

In [7]:
res

{'type': 'FeatureCollection',
 'metadata': {'generated': 1732611248000,
  'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2025-11-01',
  'title': 'USGS Earthquakes',
  'status': 200,
  'api': '1.14.1',
  'count': 0},
 'features': []}

In [5]:
len(res["features"])

0

In [24]:
from datetime import datetime
datetime.utcfromtimestamp(1732492753770/1000).strftime('%Y-%m-%dT%H:%M:%S')

'2024-11-24T23:59:13'

In [38]:
datetime.now().strftime('%Y-%m-%dT%H:%M:%S')

'2024-11-25T12:23:33'

In [44]:
import pymongo
def connect_mongo(
        mongo_client: str = "mongodb://localhost:27017/",
        mongo_db: str = "earthquake_db",
        mongo_collection: str = "earthquakes",
    ) -> tuple:
    """_summary_

    Parameters
    ----------
    mongo_client : _type_, optional
        url de connexion à la base de données MongoDB, by default "mongodb://localhost:27017/"
    mongo_db : str, optional
        _description_, by default "earthquake_db"
    mongo_collection : str, optional
        _description_, by default "earthquakes"

    Returns
    -------
    tuple
        Tuple contenant le client mongo, la db et la collection pour requêter la base de données
    """

    client = pymongo.MongoClient(mongo_client)
    db = client[mongo_db]
    collection = db[mongo_collection]

    return client, db, collection

In [46]:
client, db, collection = connect_mongo(mongo_db="superheroes", mongo_collection="info")

In [47]:
type(collection)

pymongo.synchronous.collection.Collection

In [69]:
type(client)

pymongo.synchronous.mongo_client.MongoClient

In [49]:
result = collection.aggregate([
    {"$group": {"_id": None, "maxScore": {"$max": "$weight(kg)"}}}
])

In [51]:
max_value = next(result, None)
if max_value:
    print("Valeur maximale :", max_value["maxScore"])
else:
    print("Aucun document trouvé.")

Valeur maximale : 900.0


In [ ]:
import pandas as pd

In [58]:
import pandas as pd
data=res["features"][0]
print(data)

{'type': 'Feature', 'properties': {'mag': 1.6, 'place': '53 km NW of Toyah, Texas', 'time': 1732530093219, 'updated': 1732530409152, 'tz': None, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/tx2024xdxb', 'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=tx2024xdxb&format=geojson', 'felt': None, 'cdi': None, 'mmi': None, 'alert': None, 'status': 'automatic', 'tsunami': 0, 'sig': 39, 'net': 'tx', 'code': '2024xdxb', 'ids': ',tx2024xdxb,', 'sources': ',tx,', 'types': ',origin,phase-data,', 'nst': 17, 'dmin': 0.1, 'rms': 0.5, 'gap': 103, 'magType': 'ml', 'type': 'earthquake', 'title': 'M 1.6 - 53 km NW of Toyah, Texas'}, 'geometry': {'type': 'Point', 'coordinates': [-104.18, 31.667, 5.5381]}, 'id': 'tx2024xdxb'}


In [61]:
res["features"][0]["properties"]

{'mag': 1.6,
 'place': '53 km NW of Toyah, Texas',
 'time': 1732530093219,
 'updated': 1732530409152,
 'tz': None,
 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/tx2024xdxb',
 'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=tx2024xdxb&format=geojson',
 'felt': None,
 'cdi': None,
 'mmi': None,
 'alert': None,
 'status': 'automatic',
 'tsunami': 0,
 'sig': 39,
 'net': 'tx',
 'code': '2024xdxb',
 'ids': ',tx2024xdxb,',
 'sources': ',tx,',
 'types': ',origin,phase-data,',
 'nst': 17,
 'dmin': 0.1,
 'rms': 0.5,
 'gap': 103,
 'magType': 'ml',
 'type': 'earthquake',
 'title': 'M 1.6 - 53 km NW of Toyah, Texas'}

Au dessus, on veut garder ```mag```, ```place```, ```time```, ```type```, ```nst```, ```dmin```, ```sig```, ```magType```

In [66]:
res["features"][0]["geometry"]

{'type': 'Point', 'coordinates': [-104.18, 31.667, 5.5381]}

In [67]:
final_dict = {
    "mag": res["features"][0]["properties"]["mag"],
    "place": res["features"][0]["properties"]["place"],
    "time": res["features"][0]["properties"]["time"],
    "type": res["features"][0]["properties"]["type"],
    "nst": res["features"][0]["properties"]["nst"],
    "dmin": res["features"][0]["properties"]["dmin"],
    "sig": res["features"][0]["properties"]["sig"],
    "magType": res["features"][0]["properties"]["magType"],
    "geometryType" : res["features"][0]["geometry"]["type"],
    "coordinates" : res["features"][0]["geometry"]["coordinates"]
}

In [68]:
final_dict

{'mag': 1.6,
 'place': '53 km NW of Toyah, Texas',
 'time': 1732530093219,
 'type': 'earthquake',
 'nst': 17,
 'dmin': 0.1,
 'sig': 39,
 'magType': 'ml',
 'geometryType': 'Point',
 'coordinates': [-104.18, 31.667, 5.5381]}